In [3]:
### PLEASE COPY NOTEBOOKS TO YOUR FOLDERS TO PREVENT COMMIT CONFLICTS
from dcicutils import ff_utils
from functions.notebook_functions import *
import json

# get key from keypairs.json
my_env = 'data'
my_key = get_key('koray_data')
schema_name = get_schema_names(my_key) 
print('WORKING ON', my_key['server'], '\n')

find_linked = ['11f207be-ebc4-4622-8b42-02e7841d17db']
store, uuids = ff_utils.expand_es_metadata(find_linked, key = my_key, add_pc_wfr = True, ignore_field=['references', 'attachments'])

for key in store:
    print(key, len(store[key]))
print(len([i['uuid'] for key in store for i in store[key]]))

WORKING ON https://data.4dnucleome.org 

static_section 1
file_processed 23
file_reference 3
file_fastq 20
experiment_set_replicate 1
organism 1
user 17
antibody 4
lab 9
workflow_run_awsem 44
software 12
experiment_seq 10
vendor 4
ontology 2
workflow 6
individual_human 1
protocol 2
biosource 1
ontology_term 10
award 8
biosample_cell_culture 2
file_format 7
quality_metric_chipseq 11
biosample 2
quality_metric_fastqc 16
target 4
221


In [5]:
#### This part should only run once!

transfer_env = 'local'
transfer_key = get_key('local')

#### This part should only run once!

# if the item exist in the target, should it overwrite it (will include user/award etc)
overwrite_existing = True

# reverse lookup dictionary for schema names
rev_schema_name = {}
for key, name in schema_name.items():
    rev_schema_name[name] = schema_name[key]

my_types = [i for i in ORDER if i in store.keys()]

second_round_items = {}

# Round I - only put the required - skip if exists already
for a_type in my_types:
    print(a_type)
    obj_type = rev_schema_name[a_type]
    # find required field
    schema_info = ff_utils.get_metadata('/profiles/{}.json'.format(a_type), key=transfer_key)
    req_fields = schema_info['required']
    ids = schema_info['identifyingProperties']
    first_fields = list(set(req_fields+ids))
    remove_existing_items = []
    
    print(len(store[a_type]), 'items exist on source')
    posted = 0
    skip_exist = 0
    for an_item in store[a_type]:
        exists = False
        try:
            # TODO check with all identifiers
            existing = ff_utils.get_metadata(an_item['uuid'], key=transfer_key)
            exists = True
        except:
            exists = False
        # skip the items that exists, if overwrite is not allowed, they them out from patch list
        if exists and existing:
            skip_exist += 1
            if not overwrite_existing:
                remove_existing_items.append(an_item['uuid'])
            # print("{} {} can not post existing item".format(obj_type, an_item['uuid']))
            continue
        posted += 1
        post_first = {key:value for (key,value) in an_item.items() if key in first_fields}
        ff_utils.post_metadata(post_first, obj_type, key = transfer_key)
   
    second_round_items[a_type] = [i for i in store[a_type] if i['uuid'] not in remove_existing_items]
    print(posted, 'items posted,', skip_exist, 'existing items skipped')
    print(len(second_round_items[a_type]), 'items will be patched in second round')
    print()
    

user
17 items exist on source
0 items posted, 17 existing items skipped
17 items will be patched in second round

award
8 items exist on source
0 items posted, 8 existing items skipped
8 items will be patched in second round

lab
9 items exist on source
0 items posted, 9 existing items skipped
9 items will be patched in second round

static_section
1 items exist on source
0 items posted, 1 existing items skipped
1 items will be patched in second round

ontology
2 items exist on source
0 items posted, 2 existing items skipped
2 items will be patched in second round

ontology_term
10 items exist on source
0 items posted, 10 existing items skipped
10 items will be patched in second round

file_format
7 items exist on source
0 items posted, 7 existing items skipped
7 items will be patched in second round

organism
1 items exist on source
0 items posted, 1 existing items skipped
1 items will be patched in second round

target
4 items exist on source
0 items posted, 4 existing items skipped


In [8]:
# Round II - patch the rest of the metadata
for a_type in my_types:
    obj_type = rev_schema_name[a_type]
    if not second_round_items[a_type]:
        print(a_type, '- no items to patch')
        continue 
    for an_item in second_round_items[a_type]:
        if a_type == 'file_fastq':
            if 'extra_files' in an_item:
                del an_item['extra_files']
        ff_utils.patch_metadata(an_item, obj_id = an_item['uuid'], key = transfer_key)
    print(a_type, '- patched')

user - patched
award - patched
lab - patched
static_section - patched
ontology - patched
ontology_term - patched
file_format - patched
organism - patched
target - patched
vendor - patched
protocol - patched
biosample_cell_culture - patched
individual_human - patched
biosource - patched
antibody - patched
biosample - patched
quality_metric_fastqc - patched
quality_metric_chipseq - patched
file_fastq - patched
file_processed - patched
file_reference - patched
experiment_seq - patched
experiment_set_replicate - patched
software - patched


Exception: Bad status code for PATCH request for http://localhost:8000/1f04547b-7d26-43a4-888d-2d8a4595f513: 422. Reason: {'code': 422, 'status': 'error', 'description': 'Failed validation', 'errors': [{'name': ['schema_version'], 'location': 'body', 'description': "request method 'PATCH' is not one of "}, {'name': ['category'], 'location': 'body', 'description': "'Other' is not of type 'array'"}, {'name': [], 'location': 'body', 'description': "Additional properties are not allowed ('data_types', 'workflow_diagram', 'workflow_type' were unexpected)"}], 'title': 'Unprocessable Entity', '@type': ['ValidationFailure', 'Error']}